In [1]:
import pandas as pd
import os
import numpy as np
import json

In [2]:
def get_yearly_visitors(path, year, if_target):
    data = os.listdir(path)
    year = str(year)
    data = [path+'/'+i for i in data if year in i]
    df = []
    for i in data:
        df.append(pd.read_csv(i))
    df = pd.concat(df)
    if if_target == 1:
        df = df.drop(columns = ['Unnamed: 0']).rename(columns={"Unnamed: 0.1": "cbg"})
        df = df.groupby('cbg').sum().reset_index()
        df['total_visitors'] = df.drop(columns = ['cbg']).apply(lambda x: x.sum(),axis=1)
    if if_target == 0:
        df = df.rename(columns={"Unnamed: 0": "cbg"})
        df = df.groupby('cbg').sum().reset_index()
        df['total_visitors'] = df.drop(columns = ['cbg']).apply(lambda x: x.sum(),axis=1)
    return df

In [3]:
years = [2018, 2019, 2020]
save = '/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/GOOGLEDRIVES/data'
for year in years:
    temp_target = get_yearly_visitors('/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/filtered_target_0201_matrix_aggregated', year, 1)
    temp_competitors = get_yearly_visitors('/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/filtered_competitors_0201_matrix', year, 0)
    temp_all = pd.merge(temp_target.drop(columns = 'total_visitors'), temp_competitors.drop(columns = 'total_visitors'))
    temp_all['total_visitors'] = temp_all.drop(columns = ['cbg']).apply(lambda x: x.sum(), axis=1)
    temp_target.to_csv(save+'/'+'Target_total_visitors_'+str(year)+'.csv')
    temp_competitors.to_csv(save+'/'+'Competitors_total_visitors_'+str(year)+'.csv')
    temp_all.to_csv(save+'/'+'All_stores_total_visitors_'+str(year)+'.csv')

In [4]:
def get_geo_data(info_path):
    temp = pd.read_csv(info_path, encoding = 'latin-1')
    temp['geo'] = temp.apply(lambda row: (row.latitude, row.longitude),axis=1)
    temp = temp[['safegraph_place_id', 'geo']]
    return temp

In [5]:
square_data = pd.read_csv('/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/sample_data/SafeGraphPlacesGeoSupplementSquareFeet.csv')

In [6]:
info_path_target = '/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/NYC_target/NYC_target_info.csv'
info_path_competitors = '/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/competitors_1030/places_info_nyc.csv'
target_info_geo = get_geo_data(info_path_target)
competitors_info_geo = get_geo_data(info_path_competitors)
all_info_geo = target_info_geo.append(competitors_info_geo, ignore_index=True)

In [7]:
target_ids = target_info_geo.safegraph_place_id.tolist()
competitor_ids = list(temp_competitors.columns[1:-1])
all_ids = target_ids + competitor_ids

In [8]:
square_data_part = square_data[square_data['safegraph_place_id'].isin(all_ids)]
all_info_square = square_data_part[['safegraph_place_id', 'area_square_feet']]

In [9]:
all_info = pd.merge(all_info_geo, all_info_square, on='safegraph_place_id')

In [10]:
all_info.to_csv('/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/GOOGLEDRIVES/data/all_info_before_aggregation.csv')

In [11]:
list_517E = ['sg:b92ac1661ff14e78b25a99515d2370c5','sg:507b345e2e2445f5b4edaf6fa1b01b0d', 'sg:fcf8054ca0eb4daa8fa77bd2ea68987f']
list_4024_College = ['sg:37cd539e6ba84eb898b7f979a38e08cc','sg:da1cac5e846a4c39ab873c53a240da32', 'sg:83ad2c11ae2f4810a3aa10155a1d7e9e']
list_40W = ['sg:3ee2fa14ed7841c88080d239f4aa7292', 'sg:a5570cc64afd4f12b4bc7a8b68774922']
removed_targets = list_517E + list_4024_College + list_40W

In [12]:
all_info = all_info.append({'safegraph_place_id':'aggregated_517E','geo':(40.795408, -73.93100799999999), 'area_square_feet':169356},
                ignore_index=True)
all_info = all_info.append({'safegraph_place_id':'aggregated_4024_College','geo':(40.757749, -73.834744), 'area_square_feet':214735},
                ignore_index=True)
all_info = all_info.append({'safegraph_place_id':'aggregated_40W','geo':(40.872776, -73.907558), 'area_square_feet':183593},
                ignore_index=True)
all_info = all_info[~all_info.safegraph_place_id.isin(removed_targets)]

In [13]:
all_info.to_csv('/Volumes/xuyilun/课题/Alex Sandy Pentland/safegraph/GOOGLEDRIVES/data/all_info_after_aggregation.csv')

In [14]:
all_info

,safegraph_place_id,geo,area_square_feet
0,sg:81aae721ea114787a6e105b349eeb400,"(40.768158, -73.96154)",19853
1,sg:1ac978d80ab84d4aa1d9f7148d1fafb8,"(40.783772, -73.833733)",68087
2,sg:72ea28e3e1a54cc0849b69121b98e1cd,"(40.684647, -73.976486)",126053
3,sg:084ab2c726784ab490b72144d44d4ee8,"(40.631371, -73.946744)",40818
4,sg:eb96ed984b274a57ae29762d73738d99,"(40.607664, -73.959466)",18706
...,...,...,...
288,sg:84a8615ac25b455bb1d01ec4c93cdd1f,"(40.583277, -74.165371)",96668
289,sg:ede0c3b8b3a143848181ee2430390072,"(40.659248, -73.908321)",5885
290,aggregated_517E,"(40.795408, -73.93100799999999)",169356
291,aggregated_4024_College,"(40.757749, -73.834744)",214735
